# Libraries

In [2]:
# Import packages
# Performance tracking
import time
from datetime import timedelta
import datetime
import psutil # for memory checks
from psutil._common import bytes2human
import sys
import gc

# Data processing
import pandas as pd
import geopandas as gpd
import os
import shapely                 #needed to set geopandas geometry 
from shapely.wkt import loads  #needed to set geopandas geometry
import rasterio as rio # for rasters
import rasterio.mask
from rasterio import features
import numpy as np
import dask.array as da # Package added to sds2024 environment for chunking and paralllel processing of large files

In [3]:
# General Paths
preppedDat_path = 'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/'
rawDat_path = 'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/raw/'
output_path = 'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/'
interimFiles_path = 'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/'
workFiles_path = 'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/work files/'

# Functions
## Poly Rasterization
### Iterable function

In [1]:
# Created 31/01
# Last edit 4/03 adding all_touched arg (True or False)

def polyrast(poly_path, raster_path, logpath, all_touched):
    start = time.perf_counter()
    logstart = f'\nPolyrast Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logstart)
    with open(logpath, 'a') as logfile:
        logfile.writelines([logstart,'\n'])
    # Getting file name for output naming
    filename = os.path.basename(poly_path)
    filename_noExt = os.path.splitext(filename)[0]
    ext = os.path.splitext(filename)[1]
    polyrast_write_path = f'{interimFiles_path}polyrast/{filename_noExt}.tif'
    # Check if file exists
    if not os.path.exists(polyrast_write_path):
        logexist = f'File does not exist: {polyrast_write_path}. \n Processing....'
        print (logexist)
        # Read the reference raster
        logref = f'Read ref raster Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n from: {raster_path}'
        print(logref)
        with rio.open(raster_path) as src:
            # Read poly layer
            logrd = f'Read poly Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n from: {poly_path}'
            print(logrd)
            vector = gpd.read_file(poly_path)
        
            # Rasterize poly
            #https://pygis.io/docs/e_raster_rasterize.html
            geom = [shapes for shapes in vector.geometry]
            logras = f'Rasterize poly Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
            print(logras)
            # Rasterize vector using the shape and coordinate system of the raster
            rasterized = features.rasterize(geom, out_shape = src.shape, fill = 0, out = None, transform = src.transform, all_touched = all_touched, default_value = 1, dtype = None) # All touched = False --> only pixels fully inside the geom
        
            # Write to file
            logwri = f'Writing rasterized poly to file Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
            print(logwri)
            with rio.open(
            polyrast_write_path, 'w',
            driver='GTiff',
            height=src.shape[0],
            width=src.shape[1],
            count=1,
            dtype=rasterized.dtype,
            crs=src.crs,
            transform=src.transform,
            compress='DEFLATE'
            ) as dst:
                dst.write(rasterized, 1)
        with open(logpath, 'a') as logfile:
            logfile.writelines([logexist,'\n',logref,'\n',logrd,'\n',logras,'\n',logwri,'\n'])
    else:
        logexist = f'File already exists: {polyrast_write_path}. \n Skipping....'
        print (logexist)
        with open(logpath, 'a') as logfile:
            logfile.writelines([logexist,'\n'])

    # Clearing variables for memory
    try:
        del vector
    except:
        print("('vector' var could not be deleted, likely doesn't exist)")
    try:
        del geom
    except:
        print("('geom' var could not be deleted, likely doesn't exist)")
    try:
        del rasterized
    except:
        print("('rasterized' var could not be deleted, likely doesn't exist)")
    try:
        del src
    except:
        print("('src' var could not be deleted, likely doesn't exist)")
    
    logend = f'Polyrast End {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logend)
    end = time.perf_counter()
    elapsed_time = end - start
    # Convert the elapsed time to a timedelta object 
    elapsed_time_str = str(timedelta(seconds=elapsed_time))
    logrun = f'Polyras Code block run in {elapsed_time_str}'
    print(logrun)
    with open(logpath, 'a') as logfile:
        logfile.writelines([logend,'\n', logrun])
    return polyrast_write_path

## Memory usage check

In [15]:
## Check memory usage
logram = f'RAM memory % used:{psutil.virtual_memory()[2]} --- RAM Used (GB):{bytes2human(psutil.virtual_memory()[3])}\nSwap memory used {psutil.swap_memory().percent}% --- {bytes2human(psutil.swap_memory().used)}'
print(logram)
## Code from:
## https://stackoverflow.com/questions/40993626/list-memory-usage-in-ipython-and-jupyter
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
print(f'Memory use: {sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:10]}')

RAM memory % used:51.0 --- RAM Used (GB):8.717692928
Swap memory used 19.2% --- 9.2G 
Memory use: [('timedelta', 432), ('bytes2human', 160), ('loads', 160), ('open', 160), ('interimFiles_path', 157), ('preppedDat_path', 149), ('rawDat_path', 140), ('output_path', 137), ('logram', 126), ('da', 72)]


# Layer Prep

## General file processing
### Slope

#### Unzipping DTM ASCII files

In [ ]:
# Need to get DTM first
# OS Panorama is only available as an ascii grid for raster files, which is split in thousands of individual zips. Need to extract those for batch processing and merging in QGIS.
# Extract all zips nested in all subfolders: 
# (Thanks Edge AI assistant for this one)
import zipfile

# Specify the directory path
directory_path = 'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/raw/terr50_gagg_gb'

# Function to unzip all files in a directory
def unzip_all_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.zip'):
                file_path = os.path.join(root, file)
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    # Extract all contents to the same directory as the zip file
                    zip_ref.extractall(root)
                print(f'Extracted: {file_path}')

# Call the function
unzip_all_files(directory_path)


#### Scaling values and saving as int16

In [4]:
start = time.perf_counter()  # performance counter

# Paths
read_path = preppedDat_path + 'OS Terrain 10m slope bicubic deflate float32.tif'
write_path = preppedDat_path + 'OS Terrain 10m slope bicubic deflate scale_100 uint16.tif'

# Open raster file
with rio.open(read_path) as src:
    # Read raster as dask array
    raster_data = da.from_array(src.read(1), chunks=(50))  
    
    # Multiply values by 100 and convert to integers
    processed_data = (raster_data * 100).astype('uint16')  
    
    # Write to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=src.shape[0],
        width=src.shape[1],
        count=1,
        dtype='uint16',  # Save as 16-bit unsigned integer
        crs=src.crs,
        transform=src.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

end = time.perf_counter()  # performance counter
elapsed_time = end - start
# Convert the elapsed time to a timedelta object
elapsed_time_str = str(timedelta(seconds=elapsed_time))
print(f'Code block run in {elapsed_time_str}')
del raster_data
del processed_data
del src
del dst
gc.collect()

C:\Users\roro_\miniconda3\envs\sds2024_IGS\Lib\site-packages\dask\array\chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


Code block run in 0:40:10.188348


0

There are bugs in the output, for some reason. Working with float32 for now.

## Poly data exploration

### Airports

In [3]:
# Check airport data
airports_path = f'{rawDat_path}Airports/hotosm_gbr_airports_points_gpkg.gpkg'
air_gdf = gpd.read_file(airports_path)

In [4]:
len(air_gdf)

5701

In [5]:
air_gdf['aeroway'].unique()

array(['helipad', 'windsock', 'gate', 'holding_position', 'threshold',
       'parking_position', 'airstrip', 'aerodrome', 'navigationaid',
       'tower', 'terminal', 'ndb', 'signal_square', 'radar', 'yes',
       'turn_pad', 'check_in_counter', None, 'control_tower', 'fuel',
       'heliport', 'arresting_gear', 'check-in desks', 'anemometer',
       'aircraft_crossing', 'paragliding_takeoff_site'], dtype=object)

In [6]:
air_gdf.loc[air_gdf['aeroway']=='yes']

,name,name:en,aeroway,building,emergency,emergency:helipad,operator:type,capacity:persons,addr:full,addr:city,source,osm_id,osm_type,geometry
742,Zone C,None,yes,None,None,None,None,None,None,None,None,4485745431,nodes,POINT (-0.17693 51.16067)
1724,Zone H,None,yes,None,None,None,None,None,None,None,None,4485745426,nodes,POINT (-0.17717 51.16149)
1726,Zone G,None,yes,None,None,None,None,None,None,None,None,4485745424,nodes,POINT (-0.17709 51.16109)
1727,Zone F,None,yes,None,None,None,None,None,None,None,None,4485745425,nodes,POINT (-0.17707 51.16103)
1728,Zone E,None,yes,None,None,None,None,None,None,None,None,4485745429,nodes,POINT (-0.17700 51.16088)
2958,Zone D,None,yes,None,None,None,None,None,None,None,None,4485745430,nodes,POINT (-0.17699 51.16082)
4041,Zone A,None,yes,None,None,None,None,None,None,None,None,4485745433,nodes,POINT (-0.17684 51.16044)
4345,Oversized Baggage 1,None,yes,None,None,None,None,None,None,None,None,4485745432,nodes,POINT (-0.17690 51.16061)


In [12]:
select_air = air_gdf.loc[air_gdf['aeroway'].isin(['helipad',
       'airstrip', 'aerodrome',
       'heliport'
        ])]
select_air

,name,name:en,aeroway,building,emergency,emergency:helipad,operator:type,capacity:persons,addr:full,addr:city,source,osm_id,osm_type,geometry
0,Snarehill Hall helipad,None,helipad,None,None,None,None,None,None,None,None,12246600491,nodes,POINT (0.78081 52.41604)
19,Cottage Farm Airstrip,None,airstrip,None,None,None,None,None,None,Bloxholm,None,8002936047,nodes,POINT (-0.42789 53.05560)
20,Home Farm Private Airstrip,None,airstrip,None,None,None,None,None,None,None,None,8008822001,nodes,POINT (-1.71953 52.10776)
21,Bonnington Airstrip,None,airstrip,None,None,None,None,None,None,None,None,8010567595,nodes,POINT (0.94806 51.05891)
22,Brickwall Farm Airstrip,None,aerodrome,None,None,None,None,None,None,None,None,8041513006,nodes,POINT (0.53236 52.09393)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5652,None,None,helipad,None,None,None,None,None,None,None,None,12444549543,nodes,POINT (-0.26820 51.31729)
5653,Port Adhair Barraigh,Barra Airport,aerodrome,None,None,None,None,None,None,Isle of Barra,wikipedia,1042032788,nodes,POINT (-7.44752 57.02501)
5666,None,None,airstrip,None,None,None,None,None,None,None,None,12456067269,nodes,POINT (-3.72891 50.50611)
5679,None,None,helipad,None,None,None,None,None,None,None,None,12462520943,nodes,POINT (-3.05094 53.58453)


In [13]:
select_air.to_file(f'{preppedDat_path}Airports/BG_OSM_HDX_Airports.gpkg')

After checking in QGIS, turns out removing all these classes only removes the big airports. Small countryside airstrips are preserved. --> **use the whole dataset**

## Rasterizations

### Woodland

In [5]:
# Woodland layer
start = time.perf_counter() # performance counter
# Extracting Woodland from LULC
# Paths
lulc_path = preppedDat_path+'UKCEH_LC_2023_10m.tif' # LULC data 
write_path = preppedDat_path+'UKCEH_10m_woodland.tif' # output file

# Open the raster file
with rio.open(lulc_path) as src:
    # Read the raster data as a dask array
    raster_data = da.from_array(src.read(1), chunks=(100, 100)) #NB: using dask to split file in chunks for multithreaded processing, as image was too big to be read directly
    
    # Perform operations on the dask array
    mask = np.isin(raster_data, [1, 2]) # Select broadleaved (1) and coniferous (2) woodland pixels
    processed_data = np.where(mask, 1, 0).astype('uint8') #Pixels out of mask set to 0, inside mask set to 1. Result as unsigned int8 data to reduce file size & memory usage (not sure it'll make a difference as original file is already 8-bit unsigned. Does seem to fix my out-of-memory issues)

    with rio.open(
    write_path, 'w',
    driver='GTiff',
    height=src.shape[0],
    width=src.shape[1],
    count=1,
    dtype='uint8',
    crs=src.crs,
    transform=src.transform,
    compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

end = time.perf_counter() # performance counter
elapsed_time = end - start
# Convert the elapsed time to a timedelta object 
elapsed_time_str = str(timedelta(seconds=elapsed_time))
print(f'Code block run in {elapsed_time_str}')

Code block run in 0:19:49.913242


### Water

In [4]:
# Water LAYER
start = time.perf_counter() # performance counter
# Extracting water from LULC
# Paths
lulc_path = preppedDat_path+'UKCEH_LC_2023_10m.tif' # LULC data 
write_path = preppedDat_path+'UKCEH_10m_water.tif' # output file

# Open the raster file
with rio.open(lulc_path) as src:
    # Read the raster data as a dask array
    raster_data = da.from_array(src.read(1), chunks=(100, 100)) #NB: using dask to split file in chunks for multithreaded processing, as image was too big to be read directly
    
    # Perform operations on the dask array
    mask = da.isin(raster_data, [13, 14]) # Select saltwater (13) and freshwater (14) pixels
    processed_data = da.where(mask, 1, 0).astype('uint8') #Pixels out of mask set to 0, inside mask set to 1. Result as unsigned int8 data to reduce file size & memory usage (not sure it'll make a difference as original file is already 8-bit unsigned)
  
    # Write the processed data back to a new raster file
    with rio.open(write_path, 'w', **src.profile) as dst:
        dst.write(processed_data.compute(), 1)

end = time.perf_counter() # performance counter
elapsed_time = end - start
# Convert the elapsed time to a timedelta object 
elapsed_time_str = str(timedelta(seconds=elapsed_time))
print(f'Code block run in {elapsed_time_str}')

Code block run in 0:19:20.002825


In [20]:
del start
gc.collect()

1823576

### Urban

In [4]:
# Urban layer
start = time.perf_counter() # performance counter
# Extracting Urban from LULC
# Paths
lulc_path = preppedDat_path+'UKCEH_LC_2023_10m.tif' # LULC data 
write_path = preppedDat_path+'UKCEH_10m_urban.tif' # output file

# Open the raster file
with rio.open(lulc_path) as src:
    # Read the raster data as a dask array
    raster_data = da.from_array(src.read(1), chunks=(100, 100)) #NB: using dask to split file in chunks for multithreaded processing, as image was too big to be read directly
    
    # Perform operations on the dask array
    mask = np.isin(raster_data, [20, 21]) # Select urban (20) and suburban (21) pixels
    processed_data = np.where(mask, 1, 0).astype('uint8') #Pixels out of mask set to 0, inside mask set to 1. Result as unsigned int8 data to reduce file size & memory usage (not sure it'll make a difference as original file is already 8-bit unsigned. Does seem to fix my out-of-memory issues)

    with rio.open(
    write_path, 'w',
    driver='GTiff',
    height=src.shape[0],
    width=src.shape[1],
    count=1,
    dtype='uint8',
    crs=src.crs,
    transform=src.transform,
    compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

end = time.perf_counter() # performance counter
elapsed_time = end - start
# Convert the elapsed time to a timedelta object 
elapsed_time_str = str(timedelta(seconds=elapsed_time))
print(f'Code block run in {elapsed_time_str}')

Code block run in 0:18:17.046125


### Protected Areas

#### UNEP-WCMC

In [4]:
# Checking data
pa_df0 = gpd.read_file(f'{rawDat_path}UNEP-WCMC and IUCN (2024) Protected Areas/WDPA_WDOECM_Mar2024_Public_GBR_shp_0/WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons.shp')
pa_df1 = gpd.read_file(f'{rawDat_path}UNEP-WCMC and IUCN (2024) Protected Areas/WDPA_WDOECM_Mar2024_Public_GBR_shp_1/WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons.shp')
pa_df2 = gpd.read_file(f'{rawDat_path}UNEP-WCMC and IUCN (2024) Protected Areas/WDPA_WDOECM_Mar2024_Public_GBR_shp_2/WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons.shp')

In [3]:
UNEP_df = gpd.read_file(f'{preppedDat_path}UNEP-WCMC_Protected_Areas_2024_merged.gpkg')
JNCC_df = gpd.read_file(f'{preppedDat_path}JNCC_Ramsar_SPA.gpkg')

In [5]:
UNEP_df

,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,layer,path,geometry
0,959.0,959,1,Lake District,Lake District,National Park,National Park,National,V,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-2.86963 54.19145, -2.87077 54..."
1,960.0,960,1,Snowdonia,Snowdonia,National Park,National Park,National,V,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-3.88655 53.27407, -3.88651 53..."
2,961.0,961,1,Yorkshire Dales,Yorkshire Dales,National Park,National Park,National,V,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-2.27162 54.06538, -2.27163 54..."
3,962.0,962,1,North York Moors,North York Moors,National Park,National Park,National,V,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-1.30412 54.32533, -1.30419 54..."
4,963.0,963,1,Peak District,Peak District,National Park,National Park,National,V,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-1.85357 53.29334, -1.85395 53..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10897,170072.0,170072,1,River Axe,River Axe,Site Of Special Scientific Interest (Uk),Site Of Special Scientific Interest (Gb),National,IV,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-2.96084 50.81692, -2.96083 50..."
10898,170073.0,170073,1,River Barle,River Barle,Site Of Special Scientific Interest (Uk),Site Of Special Scientific Interest (Gb),National,IV,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-3.82724 51.16635, -3.82717 51..."
10899,170074.0,170074,1,River Borgie,River Borgie,Site Of Special Scientific Interest (Uk),Site Of Special Scientific Interest (Gb),National,IV,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-4.26209 58.52818, -4.26246 58..."
10900,170076.0,170076,1,River Dee (Parton To Crossmichael),River Dee (Parton To Crossmichael),Site Of Special Scientific Interest (Uk),Site Of Special Scientific Interest (Gb),National,IV,Not Applicable,...,State Verified,1839,Not Reported,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-4.05856 55.01027, -4.05851 55..."


In [17]:
UNEP_df['IUCN_CAT'].unique()

array(['V', 'Not Assigned', 'II', 'IV', 'III', 'Not Applicable',
       'Not Reported', 'Ia', 'VI'], dtype=object)

In [7]:
UNEP_df['DESIG'].unique()

array(['National Park', 'National Nature Reserve',
       'UNESCO-MAB Biosphere Reserve', 'Heritage Coast',
       'World Heritage Site (natural or mixed)', 'National Scenic Area',
       'Area Of Outstanding Natural Beauty', 'Regional Park',
       'Ramsar Site, Wetland of International Importance',
       'Area Of Special Scientific Interest (Ni)', 'Local Nature Reserve',
       'Site Of Special Scientific Interest (Uk)',
       'Area Of Outstanding Natural Beauty (Ni)',
       'Special Protection Area', 'Marine Protected Area (OSPAR)',
       'Marine Conservation Zone', 'Voluntary Reserve',
       'Nature Conservation Marine Protected Area', 'Nature Reserve',
       'Wildland reserve',
       'Demonstration And Research Marine Protected Area',
       'Marine Protected Area'], dtype=object)

In [39]:
# Get all unique values of DESIG for each IUCN_CAT and convert to strings
unique_desig_per_iucn = UNEP_df.groupby('IUCN_CAT')['DESIG'].apply(lambda x: ', '.join(x.unique())) # forcing it to save as strings as the numpy array originally returned was not saving well into csv (truncated list)
# Save to file for easy retrieval
unique_desig_per_iucn.to_csv(f'{preppedDat_path}UNEP-WCMC_Protected_Areas_2024_merged_designations.csv', header=True)
# Display the result
print(unique_desig_per_iucn)

IUCN_CAT
II                          National Nature Reserve, Nature Reserve
III               National Nature Reserve, Site Of Special Scien...
IV                National Nature Reserve, Area Of Special Scien...
Ia                Ramsar Site, Wetland of International Importan...
Not Applicable    UNESCO-MAB Biosphere Reserve, World Heritage S...
Not Assigned      National Nature Reserve, Heritage Coast, Natio...
Not Reported      Ramsar Site, Wetland of International Importan...
V                 National Park, Heritage Coast, Area Of Outstan...
VI                                                   Nature Reserve
Name: DESIG, dtype: object


In [34]:
# Checking differently because the list in the csv appeared incomplete
for i in UNEP_df['IUCN_CAT'].unique():
    desigs = UNEP_df.loc[UNEP_df['IUCN_CAT']== i]['DESIG'].unique()
    print(f'Cat: {i}, {desigs}')

Cat: V, ['National Park' 'Heritage Coast' 'Area Of Outstanding Natural Beauty'
 'Area Of Outstanding Natural Beauty (Ni)' 'Nature Reserve']
Cat: Not Assigned, ['National Nature Reserve' 'Heritage Coast' 'National Scenic Area'
 'Regional Park' 'Marine Protected Area (OSPAR)']
Cat: II, ['National Nature Reserve' 'Nature Reserve']
Cat: IV, ['National Nature Reserve' 'Area Of Special Scientific Interest (Ni)'
 'Local Nature Reserve' 'Site Of Special Scientific Interest (Uk)'
 'Nature Conservation Marine Protected Area' 'Nature Reserve'
 'Wildland reserve' 'National Scenic Area' 'Marine Protected Area (OSPAR)']
Cat: III, ['National Nature Reserve' 'Site Of Special Scientific Interest (Uk)'
 'Nature Reserve']
Cat: Not Applicable, ['UNESCO-MAB Biosphere Reserve' 'World Heritage Site (natural or mixed)']
Cat: Not Reported, ['Ramsar Site, Wetland of International Importance' 'Local Nature Reserve'
 'Special Protection Area' 'Marine Protected Area (OSPAR)'
 'Marine Conservation Zone' 'Voluntary 

In [16]:
UNEP_df.loc[UNEP_df['DESIG'].str.contains('Special Protection Area', case=False, na=False)]

,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,layer,path,geometry
5140,555541782.0,555541782,1,Teesmouth and Cleveland Coast,Teesmouth and Cleveland Coast,Special Protection Area,Special Protection Area (Birds Directive),Regional,Not Reported,Not Applicable,...,State Verified,1994,GB-ENG,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-1.23370 54.76956, -1.20077 54..."
7264,555703761.0,555703761,1,Solent and Dorset Coast,Solent and Dorset Coast,Special Protection Area,Special Protection Area (Birds Directive),Regional,Not Reported,Not Applicable,...,State Verified,1994,GB-ENG,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-1.16710 50.85258, -1.16710 50..."


In [10]:
# Checking number of Ramsar sites
len(UNEP_df.loc[UNEP_df['DESIG'].str.contains('Ramsar', case=False, na=False)])
# We get 151 sites

151

In [14]:
# Checking for JNCC df Ramsar (Ramsar sites are the only ones with Status == designated, the rest is SPA)
len(JNCC_df.loc[JNCC_df['STATUS']=='Designated'])
# We get 152 sites

152

In [40]:
# Checking number of SPA
len(UNEP_df.loc[UNEP_df['DESIG'].str.contains('Special Protection Area', case=False, na=False)])
# We get 2 sites

2

In [41]:
# Checking for JNCC df SPA (Ramsar sites are the only ones with Status == designated, the rest is SPA)
len(JNCC_df.loc[JNCC_df['STATUS']!='Designated'])
# We get 270 sites

270

In [ ]:
# Other checks
UNEP_df.loc[UNEP_df['DESIG_ENG'] == 'National Nature Reserve, Wetland of International Importance, Site of Community Importance, Special Protection Area Site of Community Importance']

In [10]:
pa_df.loc[pa_df['DESIG'] == 'Special Protection Area']

,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,layer,path,geometry
5140,555541782.0,555541782,1,Teesmouth and Cleveland Coast,Teesmouth and Cleveland Coast,Special Protection Area,Special Protection Area (Birds Directive),Regional,Not Reported,Not Applicable,...,State Verified,1994,GB-ENG,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-1.23370 54.76956, -1.20077 54..."
7264,555703761.0,555703761,1,Solent and Dorset Coast,Solent and Dorset Coast,Special Protection Area,Special Protection Area (Birds Directive),Regional,Not Reported,Not Applicable,...,State Verified,1994,GB-ENG,GBR,GBR,Not Applicable,Not Applicable,WDPA_WDOECM_Mar2024_Public_GBR_shp-polygons,C:/Users/roro_/Documents/University/UG year 3/...,"MULTIPOLYGON (((-1.16710 50.85258, -1.16710 50..."


##### Rasterizing poly
Polyrast doesn't work with UNEP geometries. Creating prepped geometries here and rasterizing in QGIS (GDAL "rasterize (overwrite with fixed value)" with pre-generated constant raster of correct dimensions. Only works with a temp layer then exporting to file. Doesn't work when saving straight to file, for some reason.)

In [28]:
# Setting categories
cat1 = UNEP_df.loc[(UNEP_df['IUCN_CAT'].isin(['Ia','II','III'])) & (~UNEP_df['DESIG'].str.contains('Ramsar', case=False, na=False))] # tilde inverts the boolean series
cat2 = UNEP_df.loc[(UNEP_df['IUCN_CAT'].isin(['IV','Not Applicable''Not Assigned', 'Not Reported'])) & (~UNEP_df['DESIG'].str.contains('Special Protection Area', case=False, na=False))]
cat3 = UNEP_df.loc[(UNEP_df['IUCN_CAT'].isin(['V'])) & (~UNEP_df['DESIG'].str.contains('Area Of Outstanding Natural Beauty', case=False, na=False))]
cat4 = UNEP_df.loc[(UNEP_df['IUCN_CAT'].isin(['VI'])) | (UNEP_df['DESIG'].str.contains('Area Of Outstanding Natural Beauty', case=False, na=False))]
for i,j in enumerate([cat1, cat2, cat3, cat4]):
    j.to_file(f'{preppedDat_path}UNEP-WCMC_Protected_Areas_2024_merged_cat{i}.gpkg')

##### Merging rasters

In [3]:
write_path = f'{preppedDat_path}UNEP-WCMC_Protected_Areas_2024_merged.tif'
path1 = f'{interimFiles_path}WDPA_WDOECM_Mar2024_Public_GBR_shp_0.tif'
path2 = f'{interimFiles_path}WDPA_WDOECM_Mar2024_Public_GBR_shp_1.tif'
path3 = f'{interimFiles_path}WDPA_WDOECM_Mar2024_Public_GBR_shp_2.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 1-2
    logmsk = f'Merge 1-2\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_1 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_2) #Pixels in mask set to 1        
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    # Merging result - 3
    with rio.open(path3) as src_3:
        dask_ar_3 = da.from_array(src_3.read(1), chunks= 100)
    logmsk = f'Merge result-3\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = processed_data > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_3) #Pixels in mask set to 1       
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge result-3 complete.'
    print(logdif)
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 1-2
 Mask creation Start 2025-02-05 14:01:22
Merge Start 2025-02-05 14:01:22
2025-02-05 14:01:22 - Merge 1-2 complete.
Merge result-3
 Mask creation Start 2025-02-05 14:02:06
Merge Start 2025-02-05 14:02:06
2025-02-05 14:02:06 - Merge result-3 complete.
2025-02-05 14:17:18 - Merge final result written to file.


#### JNCC

In [16]:
# Rasterizing JNCC PAs
poly = f'{preppedDat_path}JNCC_Ramsar_SPA.gpkg'
rast = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'
logpath = f'{interimFiles_path}polyrast/PAs_JNCC_polyrast_001.txt'
polyrast(poly_path = poly, raster_path = rast, logpath = logpath)

Process Start 2025-02-03 21:28:41
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/JNCC_Ramsar_SPA.tiff. 
 Processing....

Read ref raster Start 2025-02-03 21:28:41
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_for_mcdm/UKCEH_LULC_10m.tif
Read poly Start 2025-02-03 21:28:42
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_for_mcdm/JNCC_Ramsar_SPA.gpkg
Rasterize poly Start 2025-02-03 21:29:35
Writing rasterized poly to file Start 2025-02-03 21:34:50
Process End 2025-02-03 21:41:24
Code block run in 0:12:42.255112


####  Wind Turbines Spatial Framework Scotland

In [14]:
tur_path = f'{rawDat_path}Protected Areas/Wind_Turbine_Spatial_Framework_-_Scotland/pub_wtsf.shp'
tur = gpd.read_file(tur_path)

In [17]:
tur_sel = tur.loc[tur['group_cat']=='Group 1']
tur_sel

,local_auth,group_cat,details,la_s_code,sh_src,sh_src_id,sh_date_up,geometry
4,Argyll and Bute,Group 1,Group 1 - Area where wind farms will not be ac...,S12000035,48001def-3a1e-4156-87a0-338b06831da6,4,2025-01-20,"POLYGON ((227875.160 718070.090, 227913.500 71..."
5,Argyll and Bute,Group 1,Group 1 - Area where wind farms will not be ac...,S12000035,48001def-3a1e-4156-87a0-338b06831da6,1,2025-01-20,"MULTIPOLYGON (((150127.780 682844.070, 150130...."
15,Scottish Borders,Group 1,Group 1 - Area where wind farms will not be ac...,S12000026,54f3db36-3b87-4604-bbe3-43bd7c202d51,2654,2020-06-30,"POLYGON ((323308.620 640466.140, 323310.520 64..."
22,Scottish Borders,Group 1,Group 1 - Area where wind farms will not be ac...,S12000026,54f3db36-3b87-4604-bbe3-43bd7c202d51,2286,2020-06-30,"POLYGON ((321623.060 639607.730, 321601.010 63..."
27,Scottish Borders,Group 1,Group 1 - Area where wind farms will not be ac...,S12000026,54f3db36-3b87-4604-bbe3-43bd7c202d51,6127,2020-06-30,"POLYGON ((356091.460 634898.600, 356087.170 63..."
...,...,...,...,...,...,...,...,...
686,Scottish Borders,Group 1,Group 1 - Area where wind farms will not be ac...,S12000026,54f3db36-3b87-4604-bbe3-43bd7c202d51,7490,2020-06-30,"POLYGON ((360425.450 632415.000, 360425.400 63..."
687,Scottish Borders,Group 1,Group 1 - Area where wind farms will not be ac...,S12000026,54f3db36-3b87-4604-bbe3-43bd7c202d51,1913,2020-06-30,"POLYGON ((323537.850 640416.160, 323540.980 64..."
688,Scottish Borders,Group 1,Group 1 - Area where wind farms will not be ac...,S12000026,54f3db36-3b87-4604-bbe3-43bd7c202d51,7501,2020-06-30,"POLYGON ((356846.320 634763.120, 356826.740 63..."
689,Scottish Borders,Group 1,Group 1 - Area where wind farms will not be ac...,S12000026,54f3db36-3b87-4604-bbe3-43bd7c202d51,1719,2020-06-30,"POLYGON ((321167.380 639723.680, 321166.720 63..."


In [18]:
tur_sel.to_file(f'{preppedDat_path}Wind_Turbine_Spatial_Framework_(Scot)_Cat1.gpkg')

##### Run polyrast

In [20]:
# Rasterize
poly_path = f'{preppedDat_path}Wind_Turbine_Spatial_Framework_(Scot)_Cat1.gpkg'
logpath = f'{interimFiles_path}polyrast/WTSF_Scot_polyrast_001.txt'
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'
polyrast(poly_path, raster_path, logpath)


Polyrast Start 2025-03-01 14:25:59
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/Wind_Turbine_Spatial_Framework_(Scot)_Cat1.tif. 
 Processing....
Read ref raster Start 2025-03-01 14:25:59
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/UKCEH_LC_2023_10m.tif
Read poly Start 2025-03-01 14:25:59
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/Wind_Turbine_Spatial_Framework_(Scot)_Cat1.gpkg
Rasterize poly Start 2025-03-01 14:25:59
Writing rasterized poly to file Start 2025-03-01 14:26:15
Polyrast End 2025-03-01 14:27:13
Polyras Code block run in 0:01:14.708925


'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/Wind_Turbine_Spatial_Framework_(Scot)_Cat1.tif'

### Peatland

#### Scotland

In [21]:
pl_path = f'{rawDat_path}Peatland/Carbon_And_Peatland_2016_map_1973336994081578535.gpkg'
pl_gdf = gpd.read_file(pl_path)

In [22]:
pl_gdf

,OBJECTID_1,FID_GIS_EX,PRIMARY_LA,SECONDARY,SECONDARY_,LCS_CODE,NEW_CODE,LCS88_LABE,LCS_CODE_P,NEW_CODE_P,...,MSSG_COMB,MSSG_NAME,CARBON_25,CONC_CARBO,SHAPE_LENG,CONC_CODE1,IMPORTANCE,PRIMARY1,SE_ANNO_CAD_DATA,geometry
0,1,21324,Smooth grass/rushes,150,Smooth grass/rushes,150.0,5.0,Smooth grass/rushes,150.0,5.0,...,3.2.1.,Brown earths,0,1,751.357993,1C1,0,150,b'00',"MULTIPOLYGON (((247179.484 873798.063, 247166...."
1,2,21327,Coniferous (semi-natural),73,Coniferous (semi-natural),73.0,0.0,Coniferous (semi-natural),73.0,0.0,...,4.1.6.,Peaty gleys,0,5,214.071020,50,5,73,b'',"MULTIPOLYGON (((191475.011 873158.699, 191470...."
2,3,21329,Blanket bog/peat. veg.,182,Blanket bog/peat. veg.,182.0,10.0,Blanket bog/peat. veg.,182.0,10.0,...,1.4.2.,Mineral alluvial soils,0,1,594.076360,1E,0,182,b'00',"MULTIPOLYGON (((247718.016 873393.438, 247726...."
3,4,21329,Blanket bog/peat. veg.,182,Blanket bog/peat. veg.,182.0,10.0,Blanket bog/peat. veg.,182.0,10.0,...,3.2.1.,Brown earths,0,1,72.764092,1E,0,182,b'',"MULTIPOLYGON (((247651.008 873539.259, 247635...."
4,5,21330,Blanket bog/peat. veg.,182,Blanket bog/peat. veg.,182.0,10.0,Blanket bog/peat. veg.,182.0,10.0,...,3.3.2.,Humus-iron podzols,0,1,980.577685,1E,0,182,b'00',"MULTIPOLYGON (((175016.806 872804.600, 175010...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439695,439696,22647,Improved pasture,150,Smooth grass/rushes,150.0,5.0,Smooth grass/rushes,90.0,0.0,...,4.1.4.,Noncalcareous gleys,1,1,600.278362,1A2,0,90,b'',"MULTIPOLYGON (((274589.250 875075.625, 274626...."
439696,439697,22649,Undiff. broadleaf,76,Undiff. broadleaf,76.0,0.0,Undiff. broadleaf,76.0,0.0,...,3.3.2.,Humus-iron podzols,1,1,522.587658,10,0,76,b'00',"MULTIPOLYGON (((274929.844 875133.250, 274895...."
439697,439698,22649,Undiff. broadleaf,76,Undiff. broadleaf,76.0,0.0,Undiff. broadleaf,76.0,0.0,...,4.1.4.,Noncalcareous gleys,1,1,320.013217,10,0,76,b'',"MULTIPOLYGON (((275007.058 875107.000, 275000...."
439698,439699,22650,Improved pasture,90,Improved pasture,90.0,0.0,Improved pasture,90.0,0.0,...,3.3.2.,Humus-iron podzols,1,1,539.674669,10,0,90,b'00',"MULTIPOLYGON (((276751.531 875171.625, 276647...."


In [23]:
pl_gdf['PRIMARY_LA'].unique()

array(['Smooth grass/rushes', 'Coniferous (semi-natural)',
       'Blanket bog/peat. veg.', 'Undiff. heather moor',
       'Wet heather moor', 'Cliffs', 'Other peat', 'Improved pasture',
       'Undiff. bracken', 'Water', 'Recent ploughing', 'Quarries',
       'Open canopy (young plantation)', 'Montane veg.',
       'Undiff. Nardus/Molinia', 'Undiff. low scrub', 'Undiff. broadleaf',
       'Undiff. smooth grass.', 'Dry heather moor',
       'Coniferous (plantation)', 'Arable', 'Links area - grass',
       'Recent felling', 'Smooth grass/low scrub', 'Industrial peat',
       'Wetlands', 'Undiff. mixed woodland', 'Maritime grassland',
       'Undiff. salt marsh', 'Built-up', 'Dubh lochans', 'Dune lands',
       'Snow cover', 'Cemetries', 'Caravan sites', 'No-photography',
       'Road', 'Golf course', 'Bings (area)', 'Airfield', 'Estuary', ' ',
       'Factory', 'Forestry ripping', 'Cloud cover', 'Rail',
       'Links area - heath', 'Ski tows', 'Rhododendron', 'Imp. pasture'],
      dtyp

In [24]:
pl_gdf['SECONDARY_'].unique()

array(['Smooth grass/rushes', 'Coniferous (semi-natural)',
       'Blanket bog/peat. veg.', 'Undiff. heather moor',
       'Wet heather moor', 'Cliffs', 'Other peat', 'Undiff. bracken',
       'Improved pasture', 'Water', 'Montane veg.', 'Recent ploughing',
       'Quarries', 'Open canopy (young plantation)',
       'Undiff. smooth grass.', 'Undiff. low scrub', 'Undiff. broadleaf',
       'Dry heather moor', 'Coniferous (plantation)',
       'Undiff. Nardus/Molinia', 'Arable', 'Links area - grass',
       'Recent felling', 'Smooth grass/low scrub', 'Industrial peat',
       'Wetlands', 'Undiff. mixed woodland', 'Maritime grassland',
       'Undiff. salt marsh', 'Built-up', 'Dubh lochans', 'Dune lands',
       'Snow cover', 'Cemetries', 'Caravan sites', 'No-photography',
       'Undiff. broadleaf (point)', 'Road', 'Rhododendron', 'Golf course',
       'Bings (area)', 'Airfield', 'Estuary', ' ', 'Factory',
       'Forestry ripping', 'Cloud cover', 'Rail', 'Links area - heath',
       'Sk

In [49]:
pl_gdf.loc[pl_gdf['PRIMARY_LA'].isin([
        'Blanket bog/peat. veg.', 'Other peat',
       'Undiff. Nardus/Molinia', 'Industrial peat'
        ])]['SECONDARY_'].unique()

array(['Blanket bog/peat. veg.', 'Other peat', 'Undiff. heather moor',
       'Montane veg.', 'Wet heather moor', 'Industrial peat',
       'Undiff. Nardus/Molinia', 'Improved pasture',
       'Undiff. smooth grass.', 'Smooth grass/rushes',
       'Maritime grassland', 'Undiff. low scrub', 'Undiff. bracken',
       'Dry heather moor', 'Wetlands', 'Smooth grass/low scrub',
       'Undiff. broadleaf', 'Dubh lochans', 'Coniferous (plantation)',
       'Recent ploughing', 'Undiff. mixed woodland', 'CON.(S/NAT)(P)',
       'Ski tows', 'Cliffs', 'Water (point)'], dtype=object)

In [26]:
pl_gdf.loc[pl_gdf['SECONDARY_']=='Other peat']['PRIMARY_LA'].unique()

array(['Other peat', 'Blanket bog/peat. veg.', 'Improved pasture',
       'Undiff. heather moor', 'Wet heather moor', 'Smooth grass/rushes',
       'Undiff. bracken', 'Maritime grassland'], dtype=object)

In [52]:
# Selecting based on primary and secondary category
pl_sel = pl_gdf.loc[(pl_gdf['PRIMARY_LA'].isin([
        'Blanket bog/peat. veg.', 'Other peat',
       'Undiff. Nardus/Molinia', 'Industrial peat'
        ]))|(pl_gdf['SECONDARY_'].isin([
       'Blanket bog/peat. veg.',
       'Other peat', 
       'Undiff. Nardus/Molinia', 'Industrial peat'
        ]))][['geometry', 'PRIMARY_LA', 'SECONDARY_', 'LCS88_LABE', 'MSSG_NAME', 'CARBON_25', 'CONC_CARBO', 'IMPORTANCE']] # selecting only minimal amount of columns as some of the data is causing export issues
pl_sel

,geometry,PRIMARY_LA,SECONDARY_,LCS88_LABE,MSSG_NAME,CARBON_25,CONC_CARBO,IMPORTANCE
2,"MULTIPOLYGON (((247718.016 873393.438, 247726....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Mineral alluvial soils,0,1,0
3,"MULTIPOLYGON (((247651.008 873539.259, 247635....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Brown earths,0,1,0
4,"MULTIPOLYGON (((175016.806 872804.600, 175010....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Humus-iron podzols,0,1,0
5,"MULTIPOLYGON (((177131.289 873202.361, 177124....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Peaty gleys,0,5,1
6,"MULTIPOLYGON (((176790.469 873151.795, 176782....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Peaty gleys,0,5,1
...,...,...,...,...,...,...,...,...
439624,"MULTIPOLYGON (((264989.313 877584.750, 265006....",Undiff. heather moor,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Dystrophic blanket peat,0,6,1
439625,"MULTIPOLYGON (((264576.969 876849.000, 264564....",Undiff. heather moor,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Dystrophic blanket peat,0,6,1
439640,"MULTIPOLYGON (((266445.469 875309.000, 266350....",Other peat,Other peat,Other peat,Peaty gleyed podzols,0,5,1
439655,"MULTIPOLYGON (((265381.875 876773.750, 265368....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Peaty gleyed podzols,0,5,1


In [53]:
pl_sel.to_file(f'{preppedDat_path}Scotland_2016_peat_Primary_Secondary.gpkg')

In [54]:
# Selecting only based on primary category
pl_sel_primaryOnly = pl_gdf.loc[(pl_gdf['PRIMARY_LA'].isin([
        'Blanket bog/peat. veg.', 'Other peat',
       'Undiff. Nardus/Molinia', 'Industrial peat'
        ]))][['geometry', 'PRIMARY_LA', 'SECONDARY_', 'LCS88_LABE', 'MSSG_NAME', 'CARBON_25', 'CONC_CARBO', 'IMPORTANCE']]
pl_sel_primaryOnly

,geometry,PRIMARY_LA,SECONDARY_,LCS88_LABE,MSSG_NAME,CARBON_25,CONC_CARBO,IMPORTANCE
2,"MULTIPOLYGON (((247718.016 873393.438, 247726....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Mineral alluvial soils,0,1,0
3,"MULTIPOLYGON (((247651.008 873539.259, 247635....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Brown earths,0,1,0
4,"MULTIPOLYGON (((175016.806 872804.600, 175010....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Humus-iron podzols,0,1,0
5,"MULTIPOLYGON (((177131.289 873202.361, 177124....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Peaty gleys,0,5,1
6,"MULTIPOLYGON (((176790.469 873151.795, 176782....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Peaty gleys,0,5,1
...,...,...,...,...,...,...,...,...
439619,"MULTIPOLYGON (((262949.363 876285.548, 262818....",Other peat,Other peat,Other peat,Peaty gleys,0,5,1
439620,"MULTIPOLYGON (((262221.510 876734.490, 262234....",Other peat,Other peat,Other peat,Dystrophic blanket peat,0,6,1
439640,"MULTIPOLYGON (((266445.469 875309.000, 266350....",Other peat,Other peat,Other peat,Peaty gleyed podzols,0,5,1
439655,"MULTIPOLYGON (((265381.875 876773.750, 265368....",Blanket bog/peat. veg.,Blanket bog/peat. veg.,Blanket bog/peat. veg.,Peaty gleyed podzols,0,5,1


In [55]:
pl_sel_primaryOnly.to_file(f'{preppedDat_path}Scotland_2016_peat_Primary.gpkg')

In [64]:
# Selecting secondary only
# Selecting based on primary and secondary category
pl_sel_sec = pl_gdf.loc[(pl_gdf['SECONDARY_'].isin([
       'Blanket bog/peat. veg.','Other peat', 
       'Undiff. Nardus/Molinia', 'Industrial peat'
        ]))][['geometry', 'PRIMARY_LA', 'SECONDARY_', 'LCS88_LABE', 'MSSG_NAME', 'CARBON_25', 'CONC_CARBO', 'IMPORTANCE']] # selecting only minimal amount of columns as some of the data is causing export issues
pl_sel_sec.to_file(f'{preppedDat_path}Scotland_2016_peat_Secondary.gpkg')

#### England

In [45]:
en_pl_path = f'{rawDat_path}Peatland/Peaty_Soils_Location_-4629138299390543035.gpkg'
en_pl_gdf = gpd.read_file(en_pl_path)

In [47]:
en_pl_gdf['PCLASSDESC'].unique()

array(['Deep Peaty Soils', 'Shallow Peaty Soils',
       'Soils with Peaty Pockets'], dtype=object)

In [58]:
en_peat = en_pl_gdf.loc[en_pl_gdf['PCLASSDESC'].isin(['Deep Peaty Soils', 'Shallow Peaty Soils'])]
en_peat.to_file(f'{preppedDat_path}England_Peaty_Soils_Location_Deep_Shallow.gpkg')

In [59]:
en_pockets = en_pl_gdf.loc[en_pl_gdf['PCLASSDESC'] == 'Soils with Peaty Pockets']
en_pockets.to_file(f'{preppedDat_path}England_Peaty_Soils_Location_Pockets.gpkg')

#### Wales

In [62]:
# Shapefile contains only 1 geom, just need to rasterize
poly_path = f'{rawDat_path}Peatland/peatlands_of_wales_scg8.gpkg'
logpath = f'{interimFiles_path}polyrast/Peat_Wales_polyrast_001.txt'
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'

polyrast(poly_path, raster_path, logpath)


Polyrast Start 2025-03-01 18:57:26
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/peatlands_of_wales_scg8.tif. 
 Processing....
Read ref raster Start 2025-03-01 18:57:26
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/UKCEH_LC_2023_10m.tif
Read poly Start 2025-03-01 18:57:26
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/raw/Peatland/peatlands_of_wales_scg8.gpkg
Rasterize poly Start 2025-03-01 18:57:28
Writing rasterized poly to file Start 2025-03-01 18:57:36
Polyrast End 2025-03-01 18:58:17
Polyras Code block run in 0:00:51.075921


'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/peatlands_of_wales_scg8.tif'

#### Rasterize Eng and Scot layers

In [65]:
# Rasterize
scot1 = f'{preppedDat_path}Scotland_2016_peat_Primary.gpkg'
scot2 = f'{preppedDat_path}Scotland_2016_peat_Secondary.gpkg'
eng1 = f'{preppedDat_path}England_Peaty_Soils_Location_Deep_Shallow.gpkg'
eng2 = f'{preppedDat_path}England_Peaty_Soils_Location_Pockets.gpkg'
poly_paths = [scot1, scot2, eng1, eng2]
logpath = f'{interimFiles_path}polyrast/Peat_polyrast_001.txt'
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'
for i in poly_paths:
    polyrast(i, raster_path, logpath)


Polyrast Start 2025-03-01 18:59:27
File already exists: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/Scotland_2016_peat_Primary.tif. 
 Skipping....
('vector' var could not be deleted, likely doesn't exist)
('geom' var could not be deleted, likely doesn't exist)
('rasterized' var could not be deleted, likely doesn't exist)
('src' var could not be deleted, likely doesn't exist)
Polyrast End 2025-03-01 18:59:27
Polyras Code block run in 0:00:00.008745

Polyrast Start 2025-03-01 18:59:27
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/Scotland_2016_peat_Secondary.tif. 
 Processing....
Read ref raster Start 2025-03-01 18:59:27
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/UKCEH_LC_2023_10m.tif
Read poly Start 2025-03-01 18:59:27
 from: C:/Users/ro

#### Merge similar categories across countries

In [5]:
write_path = f'{preppedDat_path}Peatland/Peatland_Lvl1_merged.tif'
path1 = f'{preppedDat_path}Peatland/England_Peaty_Soils_Location_Deep_Shallow.tif'
path2 = f'{preppedDat_path}Peatland/Scotland_2016_peat_Primary.tif'
path3 = f'{preppedDat_path}Peatland/peatlands_of_wales_scg8.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 1-2
    logmsk = f'Merge 1-2\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_1 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_2) #Pixels in mask set to 1        
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    # Merging result - 3
    with rio.open(path3) as src_3:
        dask_ar_3 = da.from_array(src_3.read(1), chunks= 100)
    logmsk = f'Merge result-3\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = processed_data > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_3) #Pixels in mask set to 1       
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge result-3 complete.'
    print(logdif)
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 1-2
 Mask creation Start 2025-03-07 15:16:36
Merge Start 2025-03-07 15:16:36
2025-03-07 15:16:36 - Merge 1-2 complete.
Merge result-3
 Mask creation Start 2025-03-07 15:17:05
Merge Start 2025-03-07 15:17:05
2025-03-07 15:17:05 - Merge result-3 complete.
2025-03-07 15:31:54 - Merge final result written to file.


In [6]:
write_path = f'{preppedDat_path}Peatland/Peatland_Lvl2_merged.tif'
path1 = f'{preppedDat_path}Peatland/Scotland_2016_peat_Secondary.tif'
path2 = f'{preppedDat_path}Peatland/England_Peaty_Soils_Location_Pockets.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 1-2
    logmsk = f'Merge 1-2\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_1 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_2) #Pixels in mask set to 1        
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 1-2
 Mask creation Start 2025-03-07 15:33:25
Merge Start 2025-03-07 15:33:26
2025-03-07 15:33:29 - Merge 1-2 complete.
2025-03-07 15:45:52 - Merge final result written to file.


#### Merge categories (for peatstats)

In [3]:
write_path = f'{preppedDat_path}Peatland/Peatland_all_Lvl.tif'
path1 = f'{preppedDat_path}Peatland/Peatland_Lvl1_merged.tif'
path2 = f'{preppedDat_path}Peatland/Peatland_Lvl2_merged.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 2-1
    logmsk = f'Merge 2-1\nMask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_1 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_2*2)     
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 2-1
Mask creation Start 2025-03-11 11:20:22
Merge Start 2025-03-11 11:20:22
2025-03-11 11:20:22 - Merge 1-2 complete.
2025-03-11 11:32:40 - Merge final result written to file.


### Agriculture

#### Modified polyrast to process selected categories only

In [18]:
# EDITED VERSION TO TAKE EXTERNALLY FILTERED GDF RATHER THAN POLY PATH
# Created 31/01
# Last edit 3/02

def cat_polyrast(poly_path, poly_select, cat_nr, raster_path, logpath):
    start = time.perf_counter()
    logstart = f'\nPolyrast Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logstart)
    with open(logpath, 'a') as logfile:
        logfile.writelines([logstart,'\n'])
    # Getting file name for output naming
    filename = os.path.basename(poly_path)
    filename_noExt = os.path.splitext(filename)[0]
    ext = os.path.splitext(filename)[1]
    polyrast_write_path = f'{interimFiles_path}polyrast/{filename_noExt}_{cat_nr}.tif'
    # Check if file exists
    if not os.path.exists(polyrast_write_path):
        logexist = f'File does not exist: {polyrast_write_path}. \n Processing....'
        print (logexist)
        # Read the reference raster
        logref = f'Read ref raster Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n from: {raster_path}'
        print(logref)
        with rio.open(raster_path) as src:
            # Read poly layer
            logrd = f'Read poly Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n from: {poly_path}'
            print(logrd)
            vector = poly_select
        
            # Rasterize poly
            #https://pygis.io/docs/e_raster_rasterize.html
            geom = [shapes for shapes in vector.geometry]
            logras = f'Rasterize poly Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
            print(logras)
            # Rasterize vector using the shape and coordinate system of the raster
            rasterized = features.rasterize(geom, out_shape = src.shape, fill = 0, out = None, transform = src.transform, all_touched = False, default_value = 1, dtype = None) # All touched = False --> only pixels fully inside the geom
        
            # Write to file
            logwri = f'Writing rasterized poly to file Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
            print(logwri)
            with rio.open(
            polyrast_write_path, 'w',
            driver='GTiff',
            height=src.shape[0],
            width=src.shape[1],
            count=1,
            dtype=rasterized.dtype,
            crs=src.crs,
            transform=src.transform,
            compress='DEFLATE'
            ) as dst:
                dst.write(rasterized, 1)
        with open(logpath, 'a') as logfile:
            logfile.writelines([logexist,'\n',logref,'\n',logrd,'\n',logras,'\n',logwri,'\n'])
    else:
        logexist = f'File already exists: {polyrast_write_path}. \n Skipping....'
        print (logexist)
        with open(logpath, 'a') as logfile:
            logfile.writelines([logexist,'\n'])

    # Clearing variables for memory
    try:
        del vector
    except:
        print("('vector' var could not be deleted, likely doesn't exist)")
    try:
        del geom
    except:
        print("('geom' var could not be deleted, likely doesn't exist)")
    try:
        del rasterized
    except:
        print("('rasterized' var could not be deleted, likely doesn't exist)")
    try:
        del src
    except:
        print("('src' var could not be deleted, likely doesn't exist)")
    
    logend = f'Polyrast End {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logend)
    end = time.perf_counter()
    elapsed_time = end - start
    # Convert the elapsed time to a timedelta object 
    elapsed_time_str = str(timedelta(seconds=elapsed_time))
    logrun = f'Polyras Code block run in {elapsed_time_str}'
    print(logrun)
    with open(logpath, 'a') as logfile:
        logfile.writelines([logend,'\n', logrun])
    return polyrast_write_path

#### Scotland

In [65]:
poly_path = f'{rawDat_path}AgriculturalGrades/Scotland/LCA_50K_250K_union.gpkg'
poly_df = gpd.read_file(poly_path)

##### Agri land 1

In [66]:
# Agri land 1
min_cat = 0
max_cat = 2
agri1 = poly_df.loc[((poly_df['LCCODE_2']>=min_cat) & (poly_df['LCCODE_2']<max_cat))|((poly_df['LCCODE']>=min_cat) & (poly_df['LCCODE']<max_cat))]
agri1

,AREA,PERIMETER,LCCODE,Prime,Scale,Hectares,SqMid,AREA_2,PERIMETER_2,LCCODE_2,Scale_2,Water,AreaM2,geometry
28530,NaN,NaN,NaN,None,None,NaN,None,171.58200,6803.890,1.0,50K,No,1.715816e+06,"MULTIPOLYGON (((360591.031 739919.188, 360593...."
28579,NaN,NaN,NaN,None,None,NaN,None,1019.94000,36616.300,1.0,50K,No,1.019937e+07,"MULTIPOLYGON (((360922.406 738448.000, 360931...."
28678,NaN,NaN,NaN,None,None,NaN,None,69.67790,4624.500,1.0,50K,No,6.967789e+05,"MULTIPOLYGON (((351447.656 734670.375, 351452...."
32024,NaN,NaN,NaN,None,None,NaN,None,211.69700,13550.800,1.0,50K,No,2.116969e+06,"MULTIPOLYGON (((351967.906 681356.313, 351990...."
32103,NaN,NaN,NaN,None,None,NaN,None,11.69850,1625.580,1.0,50K,No,1.169845e+05,"MULTIPOLYGON (((354309.000 680094.063, 354285...."
32107,NaN,NaN,NaN,None,None,NaN,None,520.12900,16932.900,1.0,50K,No,5.201292e+06,"MULTIPOLYGON (((351010.750 679127.813, 350953...."
32110,NaN,NaN,NaN,None,None,NaN,None,4.82070,1040.530,1.0,50K,No,4.820703e+04,"MULTIPOLYGON (((354795.937 680208.313, 354806...."
32212,NaN,NaN,NaN,None,None,NaN,None,3.75999,860.705,1.0,50K,No,3.759986e+04,"MULTIPOLYGON (((315931.812 679103.250, 315940...."
32263,NaN,NaN,NaN,None,None,NaN,None,130.03200,5447.380,1.0,50K,No,1.300316e+06,"MULTIPOLYGON (((345330.656 678673.313, 345324...."
32296,NaN,NaN,NaN,None,None,NaN,None,4.74589,984.812,1.0,50K,No,4.745886e+04,"MULTIPOLYGON (((316563.187 678478.438, 316568...."


##### Agri land 2

In [67]:
# Agri land 2
min_cat = 2
max_cat = 3
agri2 = poly_df.loc[((poly_df['LCCODE_2']>=min_cat) & (poly_df['LCCODE_2']<max_cat))|((poly_df['LCCODE']>=min_cat) & (poly_df['LCCODE']<max_cat))]
agri2

,AREA,PERIMETER,LCCODE,Prime,Scale,Hectares,SqMid,AREA_2,PERIMETER_2,LCCODE_2,Scale_2,Water,AreaM2,geometry
6378,6244430.0,19109.4,2.0,Yes,250K,624.442667,Prime,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((269789.438 868851.562, 269761...."
6507,16565800.0,32116.0,2.0,Yes,250K,1656.583630,Prime,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((279004.062 864604.062, 278976...."
6556,3150330.0,14262.9,2.0,Yes,250K,315.032807,Prime,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((263460.281 864046.250, 263443...."
6598,5019910.0,13143.2,2.0,Yes,250K,501.990682,Prime,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((258785.703 862387.125, 258796...."
6921,7253020.0,15964.4,2.0,Yes,250K,725.301987,Prime,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((256071.516 857086.500, 256079...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52729,NaN,NaN,NaN,None,None,NaN,None,20.8072,1813.52,2.0,50K,No,2.080720e+05,"MULTIPOLYGON (((235955.531 636811.688, 235949...."
52731,NaN,NaN,NaN,None,None,NaN,None,25.1961,2893.49,2.0,50K,No,2.519605e+05,"MULTIPOLYGON (((239057.187 636406.938, 239041...."
52741,NaN,NaN,NaN,None,None,NaN,None,20.4794,2126.50,2.0,50K,No,2.047936e+05,"MULTIPOLYGON (((240007.562 636666.688, 240010...."
52744,NaN,NaN,NaN,None,None,NaN,None,77.4274,6270.35,2.0,50K,No,7.742740e+05,"MULTIPOLYGON (((235921.906 636638.438, 235937...."


##### Agri land 3

In [68]:
# Agri land 3
min_cat = 3
max_cat = 4
agri3 = poly_df.loc[((poly_df['LCCODE_2']>=min_cat) & (poly_df['LCCODE_2']<max_cat))|((poly_df['LCCODE']>=min_cat) & (poly_df['LCCODE']<max_cat))]
agri3

,AREA,PERIMETER,LCCODE,Prime,Scale,Hectares,SqMid,AREA_2,PERIMETER_2,LCCODE_2,Scale_2,Water,AreaM2,geometry
1647,284064000.0,331751.0,3.2,No,250K,28406.437780,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((336765.844 962665.688, 336759...."
1685,13669700.0,24257.5,3.2,No,250K,1366.967003,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((299881.500 965385.625, 299855...."
4778,5419120.0,18332.2,3.2,No,250K,541.911837,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((301017.969 913394.688, 301013...."
4936,618158.0,5189.3,3.2,No,250K,61.815803,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((284450.594 909770.125, 284605...."
4986,5781210.0,20614.9,3.2,No,250K,578.120609,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((293314.562 908474.500, 293295...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53406,NaN,NaN,NaN,None,None,NaN,None,85.74490,45549.4,3.2,50K,No,8.574495e+05,"MULTIPOLYGON (((370935.401 758921.093, 370907...."
53407,NaN,NaN,NaN,None,None,NaN,None,725.64400,45549.4,3.2,50K,No,7.256436e+06,"MULTIPOLYGON (((367030.500 758705.313, 367005...."
53408,NaN,NaN,NaN,None,None,NaN,None,85.53840,45549.4,3.2,50K,No,8.553836e+05,"MULTIPOLYGON (((368141.988 756680.693, 368169...."
53410,NaN,NaN,NaN,None,None,NaN,None,3.30451,31637.9,3.2,50K,No,4.461351e+06,"MULTIPOLYGON (((377272.594 769719.625, 377268...."


##### Agri land 4

In [69]:
# Agri land 4
min_cat = 4
max_cat = 5
agri4 = poly_df.loc[((poly_df['LCCODE_2']>=min_cat) & (poly_df['LCCODE_2']<max_cat))|((poly_df['LCCODE']>=min_cat) & (poly_df['LCCODE']<max_cat))]
agri4

,AREA,PERIMETER,LCCODE,Prime,Scale,Hectares,SqMid,AREA_2,PERIMETER_2,LCCODE_2,Scale_2,Water,AreaM2,geometry
25,2239190.0,7523.88,4.2,No,250K,223.918635,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((465232.031 1214113.625, 465173..."
36,185046.0,1796.28,4.2,No,250K,18.504584,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((460478.562 1211342.500, 460514..."
38,6706810.0,19585.50,4.2,No,250K,670.680570,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((460187.812 1210596.750, 460293..."
108,347402.0,2565.67,4.2,No,250K,34.740186,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((456909.031 1201247.125, 456880..."
216,729333.0,4492.57,4.2,No,250K,72.933315,Sq Mid,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((462375.719 1191771.875, 462471..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53390,NaN,NaN,NaN,None,None,NaN,None,20.80770,1828.220,4.1,50K,No,208076.965332,"MULTIPOLYGON (((212508.062 532174.625, 212485...."
53396,NaN,NaN,NaN,None,None,NaN,None,14.73670,1706.180,4.1,50K,No,147367.246582,"MULTIPOLYGON (((212912.625 532135.688, 212908...."
53399,NaN,NaN,NaN,None,None,NaN,None,11.42290,1251.040,4.2,50K,No,114228.507324,"MULTIPOLYGON (((211793.703 531957.000, 211785...."
53401,NaN,NaN,NaN,None,None,NaN,None,5.63755,1662.010,4.2,50K,No,56375.544922,"MULTIPOLYGON (((214351.969 531466.875, 214341...."


##### Agri land 5+

In [70]:
# Agri land 5
min_cat = 5
max_cat = 8
agri5 = poly_df.loc[((poly_df['LCCODE_2']>=min_cat) & (poly_df['LCCODE_2']<max_cat))|((poly_df['LCCODE']>=min_cat) & (poly_df['LCCODE']<max_cat))]
agri5

,AREA,PERIMETER,LCCODE,Prime,Scale,Hectares,SqMid,AREA_2,PERIMETER_2,LCCODE_2,Scale_2,Water,AreaM2,geometry
3,27771.4,1468.15,6.3,No,250K,2.777137,Uplands,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((460463.250 1219243.875, 460450..."
5,29221.4,1084.83,6.3,No,250K,2.922140,Uplands,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((460242.281 1219003.250, 460219..."
7,26633500.0,51596.50,6.3,No,250K,2663.354157,Uplands,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((461668.219 1217595.375, 461575..."
8,4863080.0,19050.60,6.3,No,250K,486.307555,Uplands,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((465491.062 1216859.625, 465609..."
12,2548910.0,19208.70,6.2,No,250K,254.890965,Uplands,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((461097.656 1214307.250, 460997..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53393,NaN,NaN,NaN,None,None,NaN,None,18.13200,5562.890,6.1,50K,No,181319.774902,"MULTIPOLYGON (((213334.766 532438.438, 213335...."
53397,NaN,NaN,NaN,None,None,NaN,None,4.64775,901.295,5.2,50K,No,46477.506836,"MULTIPOLYGON (((212912.625 532135.688, 212941...."
53400,NaN,NaN,NaN,None,None,NaN,None,3.35118,1140.130,6.1,50K,No,33511.825684,"MULTIPOLYGON (((211873.328 531991.875, 211867...."
53402,NaN,NaN,NaN,None,None,NaN,None,1.66341,558.113,5.1,50K,No,16634.142578,"MULTIPOLYGON (((214360.719 531267.375, 214346...."


##### Run rasterization

In [71]:
# Run cat_polyrast
logpath = f'{interimFiles_path}polyrast/Agri_Scotland_polyrast_001.txt'
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'
poly_list = [agri1, agri2, agri3, agri4, agri5]

for i,j in enumerate(poly_list, start = 1):
    cat_polyrast(poly_path = poly_path , poly_select = j, cat_nr = i, raster_path = raster_path, logpath = logpath)


Polyrast Start 2025-02-04 00:09:07
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/LCA_50K_250K_union_1.tif. 
 Processing....
Read ref raster Start 2025-02-04 00:09:07
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_for_mcdm/UKCEH_LULC_10m.tif
Read poly Start 2025-02-04 00:09:07
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/raw/AgriculturalGrades/Scotland/LCA_50K_250K_union.gpkg
Rasterize poly Start 2025-02-04 00:09:07
Writing rasterized poly to file Start 2025-02-04 00:09:20
Polyrast End 2025-02-04 00:10:00
Polyras Code block run in 0:00:52.784075

Polyrast Start 2025-02-04 00:10:00
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/LCA_50K_250K_union_2.tif. 
 Processing....
Rea

#### Wales

In [72]:
poly_path = f'{rawDat_path}AgriculturalGrades/Wales/wg_predictive_alc2.gpkg'
poly_df = gpd.read_file(poly_path)

In [73]:
poly_df

,gridcode,predictive,geometry
0,6.0,5,"POLYGON ((238250.000 395900.000, 238150.000 39..."
1,6.0,5,"POLYGON ((227200.000 395100.000, 227150.000 39..."
2,6.0,5,"POLYGON ((227050.000 395050.000, 227050.000 39..."
3,6.0,5,"POLYGON ((227050.000 395050.000, 227050.000 39..."
4,6.0,5,"POLYGON ((227050.000 394950.000, 227100.000 39..."
...,...,...,...
254571,6.0,5,"POLYGON ((321950.000 165000.000, 321900.000 16..."
254572,4.0,3b,"POLYGON ((322250.000 165000.000, 321950.000 16..."
254573,6.0,5,"POLYGON ((322250.000 165000.000, 322250.000 16..."
254574,5.0,4,"POLYGON ((336750.000 164650.000, 336650.000 16..."


In [74]:
poly_df['predictive'].unique()

array(['5', '3a', '4', '3b', '2', 'NA', 'U', '1'], dtype=object)

##### Agri land 1

In [75]:
# Agri land 1
cat = 1
agri1 = poly_df.loc[poly_df['predictive'].isin([f'{cat}', f'{cat}a', f'{cat}b', f'{cat}c'])]
agri1

,gridcode,predictive,geometry
1924,1.0,1,"POLYGON ((276900.000 383800.000, 276800.000 38..."
2162,1.0,1,"POLYGON ((307800.000 383100.000, 307800.000 38..."
2202,1.0,1,"POLYGON ((307050.000 383050.000, 307050.000 38..."
2224,1.0,1,"POLYGON ((307700.000 382950.000, 307700.000 38..."
2258,1.0,1,"POLYGON ((307650.000 382900.000, 307600.000 38..."
...,...,...,...
253131,1.0,1,"POLYGON ((317850.000 172750.000, 317700.000 17..."
253154,1.0,1,"POLYGON ((317550.000 172700.000, 317500.000 17..."
253157,1.0,1,"POLYGON ((318000.000 172800.000, 318100.000 17..."
253188,1.0,1,"POLYGON ((317650.000 172650.000, 317550.000 17..."


##### Agri land 2

In [76]:
# Agri land 2
cat = 2
agri2 = poly_df.loc[poly_df['predictive'].isin([f'{cat}', f'{cat}a', f'{cat}b', f'{cat}c'])]
agri2

,gridcode,predictive,geometry
20,2.0,2,"POLYGON ((239500.000 394400.000, 239400.000 39..."
25,2.0,2,"POLYGON ((239550.000 394350.000, 239500.000 39..."
31,2.0,2,"POLYGON ((239600.000 394300.000, 239550.000 39..."
45,2.0,2,"POLYGON ((239650.000 394250.000, 239600.000 39..."
58,2.0,2,"POLYGON ((239700.000 394200.000, 239650.000 39..."
...,...,...,...
254252,2.0,2,"POLYGON ((302900.000 167500.000, 302800.000 16..."
254262,2.0,2,"POLYGON ((296850.000 167800.000, 296950.000 16..."
254405,2.0,2,"POLYGON ((299400.000 166550.000, 299300.000 16..."
254423,2.0,2,"POLYGON ((299650.000 166500.000, 299650.000 16..."


##### Agri land 3

In [77]:
# Agri land 3
cat = 3
agri3 = poly_df.loc[poly_df['predictive'].isin([f'{cat}', f'{cat}a', f'{cat}b', f'{cat}c'])]
agri3

,gridcode,predictive,geometry
6,3.0,3a,"POLYGON ((226800.000 394750.000, 226750.000 39..."
10,4.0,3b,"POLYGON ((237450.000 394550.000, 237300.000 39..."
21,3.0,3a,"POLYGON ((239550.000 394400.000, 239500.000 39..."
27,4.0,3b,"POLYGON ((241400.000 394350.000, 241350.000 39..."
29,3.0,3a,"POLYGON ((238100.000 394400.000, 238200.000 39..."
...,...,...,...
254563,3.0,3a,"POLYGON ((302600.000 165700.000, 302600.000 16..."
254564,4.0,3b,"POLYGON ((307100.000 165700.000, 307000.000 16..."
254566,3.0,3a,"POLYGON ((302400.000 165650.000, 302400.000 16..."
254572,4.0,3b,"POLYGON ((322250.000 165000.000, 321950.000 16..."


##### Agri land 4

In [78]:
# Agri land 4
cat = 4
agri4 = poly_df.loc[poly_df['predictive'].isin([f'{cat}', f'{cat}a', f'{cat}b', f'{cat}c'])]
agri4

,gridcode,predictive,geometry
7,5.0,4,"POLYGON ((239400.000 394750.000, 239400.000 39..."
15,5.0,4,"POLYGON ((239650.000 394450.000, 239550.000 39..."
16,5.0,4,"POLYGON ((240900.000 394450.000, 240850.000 39..."
23,5.0,4,"POLYGON ((240200.000 394400.000, 240150.000 39..."
39,5.0,4,"POLYGON ((282000.000 323050.000, 281900.000 32..."
...,...,...,...
254555,5.0,4,"POLYGON ((332550.000 165900.000, 332400.000 16..."
254557,5.0,4,"POLYGON ((332850.000 165900.000, 332650.000 16..."
254568,5.0,4,"POLYGON ((336800.000 165350.000, 336450.000 16..."
254569,5.0,4,"POLYGON ((334850.000 165300.000, 334550.000 16..."


##### Agri land 5+

In [79]:
# Agri land 5
cat = 5
agri5 = poly_df.loc[poly_df['predictive'].isin([f'{cat}', f'{cat}a', f'{cat}b', f'{cat}c'])]
agri5

,gridcode,predictive,geometry
0,6.0,5,"POLYGON ((238250.000 395900.000, 238150.000 39..."
1,6.0,5,"POLYGON ((227200.000 395100.000, 227150.000 39..."
2,6.0,5,"POLYGON ((227050.000 395050.000, 227050.000 39..."
3,6.0,5,"POLYGON ((227050.000 395050.000, 227050.000 39..."
4,6.0,5,"POLYGON ((227050.000 394950.000, 227100.000 39..."
...,...,...,...
254558,6.0,5,"POLYGON ((303450.000 165850.000, 303300.000 16..."
254561,6.0,5,"POLYGON ((305900.000 165750.000, 305850.000 16..."
254570,6.0,5,"POLYGON ((321950.000 165050.000, 321950.000 16..."
254571,6.0,5,"POLYGON ((321950.000 165000.000, 321900.000 16..."


##### Run rasterization

In [80]:
# Run cat_polyrast
logpath = f'{interimFiles_path}polyrast/Agri_Wales_polyrast_001.txt'
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'
poly_list = [agri1, agri2, agri3, agri4, agri5]

for i,j in enumerate(poly_list, start = 1):
    cat_polyrast(poly_path = poly_path , poly_select = j, cat_nr = i, raster_path = raster_path, logpath = logpath)


Polyrast Start 2025-02-04 00:14:57
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/wg_predictive_alc2_1.tif. 
 Processing....
Read ref raster Start 2025-02-04 00:14:57
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_for_mcdm/UKCEH_LULC_10m.tif
Read poly Start 2025-02-04 00:14:57
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/raw/AgriculturalGrades/Wales/wg_predictive_alc2.gpkg
Rasterize poly Start 2025-02-04 00:14:57
Writing rasterized poly to file Start 2025-02-04 00:15:03
Polyrast End 2025-02-04 00:15:49
Polyras Code block run in 0:00:51.786063

Polyrast Start 2025-02-04 00:15:49
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/wg_predictive_alc2_2.tif. 
 Processing....
Read r

#### England

In [81]:
poly_path = f'{rawDat_path}AgriculturalGrades/England/England_ALC.gpkg'
poly_df = gpd.read_file(poly_path)

In [82]:
poly_df

,GEOGEXT,AREA,ALC_GRADE,PERIMETER,geometry
0,TG2,3712.5350,Grade 1,69726.523,"MULTIPOLYGON (((607170.000 230840.000, 607080...."
1,TG2,14694.1436,Grade 2,353091.649,"MULTIPOLYGON (((598336.077 221689.728, 598380...."
2,TG2,9606.3941,Grade 3,324491.005,"MULTIPOLYGON (((616613.668 224182.664, 616610...."
3,TG2,852.6170,Grade 4,54877.723,"MULTIPOLYGON (((623990.000 229210.000, 624020...."
4,TG2,1371.9177,Non Agricultural,95549.247,"MULTIPOLYGON (((624176.448 232699.493, 624110...."
...,...,...,...,...,...
1821,None,NaN,Urban,NaN,"MULTIPOLYGON (((91404.214 10181.069, 91480.418..."
1822,None,NaN,Grade 4,NaN,"MULTIPOLYGON (((90611.398 11793.825, 90648.821..."
1823,None,NaN,Grade 2,NaN,"MULTIPOLYGON (((89849.310 10091.251, 89905.223..."
1824,None,NaN,Grade 2,NaN,"MULTIPOLYGON (((91543.688 10769.655, 91518.380..."


In [83]:
poly_df['ALC_GRADE'].unique()

array(['Grade 1', 'Grade 2', 'Grade 3', 'Grade 4', 'Non Agricultural',
       'Urban', 'Grade 5', 'Exclusion'], dtype=object)

##### Agri land 1

In [84]:
# Agri land 1
cat = 1
agri1 = poly_df.loc[poly_df['ALC_GRADE'] == f'Grade {cat}']
agri1

,GEOGEXT,AREA,ALC_GRADE,PERIMETER,geometry
0,TG2,3712.5350,Grade 1,69726.523,"MULTIPOLYGON (((607170.000 230840.000, 607080...."
6,TG2,423.6100,Grade 1,21323.943,"MULTIPOLYGON (((618970.000 238330.000, 618900...."
16,T2V,543.3005,Grade 1,15973.449,"MULTIPOLYGON (((650202.458 298181.151, 650250...."
32,6G2,0.2891,Grade 1,685.536,"MULTIPOLYGON (((586438.880 188536.079, 586479...."
33,6G2,0.1089,Grade 1,270.058,"MULTIPOLYGON (((591421.288 187543.231, 591433...."
...,...,...,...,...,...
1717,E87,902.6960,Grade 1,47622.797,"MULTIPOLYGON (((416737.196 239744.391, 416740...."
1728,E87,569.2199,Grade 1,32552.736,"MULTIPOLYGON (((406140.000 246860.000, 406050...."
1815,None,NaN,Grade 1,NaN,"MULTIPOLYGON (((89255.670 14756.106, 89279.483..."
1816,None,NaN,Grade 1,NaN,"MULTIPOLYGON (((89065.170 14689.960, 88985.794..."


##### Agri land 2

In [85]:
# Agri land 2
cat = 2
agri2 = poly_df.loc[poly_df['ALC_GRADE'] == f'Grade {cat}']
agri2

,GEOGEXT,AREA,ALC_GRADE,PERIMETER,geometry
1,TG2,14694.1436,Grade 2,353091.649,"MULTIPOLYGON (((598336.077 221689.728, 598380...."
7,TG2,9051.9062,Grade 2,184437.084,"MULTIPOLYGON (((610053.515 241599.455, 610060...."
12,T2V,1719.9463,Grade 2,37044.450,"MULTIPOLYGON (((638912.303 296984.627, 638970...."
17,T2V,7172.2453,Grade 2,259009.258,"MULTIPOLYGON (((609723.984 269330.000, 609800...."
34,62W,260.0530,Grade 2,10505.352,"MULTIPOLYGON (((477266.348 220498.471, 477320...."
...,...,...,...,...,...
1814,None,NaN,Grade 2,NaN,"MULTIPOLYGON (((89790.129 14753.460, 89835.109..."
1818,None,NaN,Grade 2,NaN,"MULTIPOLYGON (((91058.809 9579.523, 91090.559 ..."
1823,None,NaN,Grade 2,NaN,"MULTIPOLYGON (((89849.310 10091.251, 89905.223..."
1824,None,NaN,Grade 2,NaN,"MULTIPOLYGON (((91543.688 10769.655, 91518.380..."


##### Agri land 3

In [86]:
# Agri land 3
cat = 3
agri3 = poly_df.loc[poly_df['ALC_GRADE'] == f'Grade {cat}']
agri3

,GEOGEXT,AREA,ALC_GRADE,PERIMETER,geometry
2,TG2,9606.3941,Grade 3,324491.005,"MULTIPOLYGON (((616613.668 224182.664, 616610...."
8,TG2,8711.7056,Grade 3,249911.345,"MULTIPOLYGON (((622740.819 242716.658, 622730...."
13,T2V,7236.6201,Grade 3,100310.828,"MULTIPOLYGON (((643684.105 297785.848, 643750...."
18,T2V,48286.0523,Grade 3,425273.609,"MULTIPOLYGON (((645190.000 290970.000, 645010...."
23,6G2,0.2504,Grade 3,607.908,"MULTIPOLYGON (((576643.501 189512.622, 576867...."
...,...,...,...,...,...
1743,81,37774.8472,Grade 3,471217.724,"MULTIPOLYGON (((436910.321 318515.924, 436780...."
1748,81,18679.1934,Grade 3,146661.401,"MULTIPOLYGON (((464108.931 280895.843, 464070...."
1752,81,33105.9249,Grade 3,555723.965,"MULTIPOLYGON (((399883.590 336984.663, 399910...."
1757,81,40696.5267,Grade 3,612770.571,"MULTIPOLYGON (((443564.786 262012.393, 443600...."


##### Agri land 4

In [87]:
# Agri land 4
cat = 4
agri4 = poly_df.loc[poly_df['ALC_GRADE'] == f'Grade {cat}']
agri4

,GEOGEXT,AREA,ALC_GRADE,PERIMETER,geometry
3,TG2,852.6170,Grade 4,54877.723,"MULTIPOLYGON (((623990.000 229210.000, 624020...."
9,TG2,1938.8302,Grade 4,81416.241,"MULTIPOLYGON (((621441.498 242565.364, 621410...."
14,T2V,996.7846,Grade 4,55628.099,"MULTIPOLYGON (((640061.615 291367.747, 640100...."
19,T2V,2542.1704,Grade 4,125462.465,"MULTIPOLYGON (((629158.653 286154.264, 629310...."
36,62W,516.0684,Grade 4,23969.161,"MULTIPOLYGON (((476564.243 215871.888, 476590...."
...,...,...,...,...,...
1808,None,NaN,Grade 4,NaN,"MULTIPOLYGON (((92151.080 11029.062, 92144.753..."
1809,None,NaN,Grade 4,NaN,"MULTIPOLYGON (((91954.943 12117.307, 92068.829..."
1810,None,NaN,Grade 4,NaN,"MULTIPOLYGON (((91480.418 10061.031, 91550.015..."
1811,None,NaN,Grade 4,NaN,"MULTIPOLYGON (((89904.959 10264.530, 90012.553..."


##### Agri land 5+

In [88]:
# Agri land 5
cat = 5
agri5 = poly_df.loc[poly_df['ALC_GRADE'] == f'Grade {cat}']
agri5

,GEOGEXT,AREA,ALC_GRADE,PERIMETER,geometry
41,6K,554.0400,Grade 5,39847.852,"MULTIPOLYGON (((595440.000 170400.000, 595410...."
48,6K,5.3878,Grade 5,994.672,"MULTIPOLYGON (((575057.189 168478.704, 575120...."
71,58,139.7900,Grade 5,5399.381,"MULTIPOLYGON (((350350.000 353770.000, 350240...."
79,58,126.9399,Grade 5,9882.578,"MULTIPOLYGON (((347407.784 333829.446, 347410...."
92,A7,73.2575,Grade 5,4613.164,"MULTIPOLYGON (((404156.800 94848.093, 404270.0..."
...,...,...,...,...,...
1785,None,NaN,Grade 5,NaN,"MULTIPOLYGON (((88850.592 8167.174, 88892.926 ..."
1786,None,NaN,Grade 5,NaN,"MULTIPOLYGON (((87445.652 8069.278, 87427.131 ..."
1787,None,NaN,Grade 5,NaN,"MULTIPOLYGON (((88104.466 7521.589, 88138.862 ..."
1797,None,NaN,Grade 5,NaN,"MULTIPOLYGON (((87818.980 8690.256, 87853.376 ..."


##### Run rasterization

In [89]:
# Run cat_polyrast
logpath = f'{interimFiles_path}polyrast/Agri_England_polyrast_001.txt'
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'
poly_list = [agri1, agri2, agri3, agri4, agri5]

for i,j in enumerate(poly_list, start = 1):
    cat_polyrast(poly_path = poly_path , poly_select = j, cat_nr = i, raster_path = raster_path, logpath = logpath)


Polyrast Start 2025-02-04 00:20:15
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/England_ALC_1.tif. 
 Processing....
Read ref raster Start 2025-02-04 00:20:15
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_for_mcdm/UKCEH_LULC_10m.tif
Read poly Start 2025-02-04 00:20:15
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/raw/AgriculturalGrades/England/England_ALC.gpkg
Rasterize poly Start 2025-02-04 00:20:15
Writing rasterized poly to file Start 2025-02-04 00:20:22
Polyrast End 2025-02-04 00:21:08
Polyras Code block run in 0:00:52.964614

Polyrast Start 2025-02-04 00:21:08
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/England_ALC_2.tif. 
 Processing....
Read ref raster Start 202

#### Merge countries by category

In [3]:
for i in range(1,6):
    write_path = f'{preppedDat_path}AgriGrades/UK_ALC_cat{i}.tif'
    eng_path = f'{preppedDat_path}AgriGrades/England_ALC_{i}.tif'
    sco_path = f'{preppedDat_path}AgriGrades/LCA_50K_250K_union_{i}.tif'
    wal_path = f'{preppedDat_path}AgriGrades/wg_predictive_alc2_{i}.tif'
    
    with rio.open(eng_path) as src_eng:
        dask_ar_eng = da.from_array(src_eng.read(1), chunks= 100)
    
    with rio.open(sco_path) as src_sco:
        dask_ar_sco = da.from_array(src_sco.read(1), chunks= 100)
    
        # Merging eng-sco
        logmsk = f'Merge Eng-Sco\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
        print(logmsk)
        mask = dask_ar_eng > 0 # Make a boolean array out of non-null raster pixels
        logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
        print(logdifst)
        processed_data = da.where(mask, 1, dask_ar_sco) #Pixels in mask set to 1        
        logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge cat {i} complete.'
        print(logdif)
        # Merging result - wal
        with rio.open(wal_path) as src_wal:
            dask_ar_wal = da.from_array(src_wal.read(1), chunks= 100)
        logmsk = f'Merge result-Wal\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
        print(logmsk)
        mask = processed_data > 0 # Make a boolean array out of non-null raster pixels
        logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
        print(logdifst)
        processed_data = da.where(mask, 1, dask_ar_wal) #Pixels in mask set to 1       
        logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge cat {i} complete.'
        print(logdif)
        # Write result to file
        with rio.open(
            write_path, 'w',
            driver='GTiff',
            height=processed_data.shape[0],
            width=processed_data.shape[1],
            count=1,
            dtype=processed_data.dtype,
            crs=src_eng.crs,
            transform=src_eng.transform,
            compress='DEFLATE'
        ) as dst:
            dst.write(processed_data.compute(), 1)

        logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge cat {i} written to file.'
        print(logwri)

Merge Eng-Sco
 Mask creation Start 2025-02-05 00:44:33
Merge Start 2025-02-05 00:44:33
2025-02-05 00:44:33 - Merge cat 1 complete.
Merge result-Wal
 Mask creation Start 2025-02-05 00:45:00
Merge Start 2025-02-05 00:45:00
2025-02-05 00:45:00 - Merge cat 1 complete.
2025-02-05 00:57:58 - Merge cat 1 written to file.
Merge Eng-Sco
 Mask creation Start 2025-02-05 00:58:59
Merge Start 2025-02-05 00:58:59
2025-02-05 00:59:04 - Merge cat 2 complete.
Merge result-Wal
 Mask creation Start 2025-02-05 00:59:39
Merge Start 2025-02-05 00:59:39
2025-02-05 00:59:39 - Merge cat 2 complete.
2025-02-05 01:13:24 - Merge cat 2 written to file.
Merge Eng-Sco
 Mask creation Start 2025-02-05 01:14:18
Merge Start 2025-02-05 01:14:18
2025-02-05 01:14:22 - Merge cat 3 complete.
Merge result-Wal
 Mask creation Start 2025-02-05 01:14:48
Merge Start 2025-02-05 01:14:48
2025-02-05 01:14:48 - Merge cat 3 complete.
2025-02-05 01:27:51 - Merge cat 3 written to file.
Merge Eng-Sco
 Mask creation Start 2025-02-05 01:28:

#### Merge all categories (for agristats)

Doing all 5 layers at once runs out of memory. So does QGIS when using conditional raster operations (addition/multiplication works fine but I'm getting overlap issues, and also it saves in 32 bit format so needs extra recompression after)  
--> Doing it in several steps here

In [3]:
# Layer 4 atop of 5
write_path = f'{preppedDat_path}Agrigrades/UK_ALC_4-5.tif'
path1 = f'{preppedDat_path}Agrigrades/UK_ALC_cat5.tif'
path2 = f'{preppedDat_path}Agrigrades/UK_ALC_cat4.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 2-1
    logmsk = f'Merge 2-1\nMask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_2 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 4, dask_ar_1*5)     
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 2-1
Mask creation Start 2025-03-11 12:46:38
Merge Start 2025-03-11 12:46:38
2025-03-11 12:46:38 - Merge 1-2 complete.
2025-03-11 12:57:37 - Merge final result written to file.


In [4]:
# Layer 2 atop of 3
write_path = f'{preppedDat_path}Agrigrades/UK_ALC_2-3.tif'
path1 = f'{preppedDat_path}Agrigrades/UK_ALC_cat3.tif'
path2 = f'{preppedDat_path}Agrigrades/UK_ALC_cat2.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 2-1
    logmsk = f'Merge 2-1\nMask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_2 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 2, dask_ar_1*3)     
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 2-1
Mask creation Start 2025-03-11 12:58:37
Merge Start 2025-03-11 12:58:37
2025-03-11 12:58:40 - Merge 1-2 complete.
2025-03-11 13:09:06 - Merge final result written to file.


In [5]:
# Layer 2-3 atop of 4-5
write_path = f'{preppedDat_path}Agrigrades/UK_ALC_2-3-4-5.tif'
path1 = f'{preppedDat_path}Agrigrades/UK_ALC_4-5.tif'
path2 = f'{preppedDat_path}Agrigrades/UK_ALC_2-3.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 2-1
    logmsk = f'Merge 2-1\nMask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_2 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, dask_ar_2, dask_ar_1)     
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 2-1
Mask creation Start 2025-03-11 13:09:57
Merge Start 2025-03-11 13:09:58
2025-03-11 13:10:03 - Merge 1-2 complete.
2025-03-11 13:20:40 - Merge final result written to file.


In [4]:
# Layer 1 atop of 2-3-4-5
write_path = f'{preppedDat_path}Agrigrades/UK_ALC_all_cat.tif'
path1 = f'{preppedDat_path}Agrigrades/UK_ALC_2-3-4-5.tif'
path2 = f'{preppedDat_path}Agrigrades/UK_ALC_cat1.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 2-1
    logmsk = f'Merge 2-1\nMask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_2 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, dask_ar_2, dask_ar_1)     
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 2-1
Mask creation Start 2025-03-11 14:06:08
Merge Start 2025-03-11 14:06:08
2025-03-11 14:06:08 - Merge 1-2 complete.
2025-03-11 14:29:41 - Merge final result written to file.


### Substations

In [19]:
poly_path = f'{preppedDat_path}substations_Lovett2022.gpkg'
logpath = f'{interimFiles_path}polyrast/Substations_polyrast_001.txt'
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'

In [20]:
polyrast(poly_path, raster_path, logpath)


Polyrast Start 2025-02-06 00:06:49
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/substations_Lovett2022.tif. 
 Processing....
Read ref raster Start 2025-02-06 00:06:49
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/UKCEH_LC_2023_10m.tif
Read poly Start 2025-02-06 00:06:49
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/substations_Lovett2022.gpkg
Rasterize poly Start 2025-02-06 00:06:51
Writing rasterized poly to file Start 2025-02-06 00:07:04
Polyrast End 2025-02-06 00:07:45
Polyras Code block run in 0:00:55.775472


'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/substations_Lovett2022.tif'

### Rasterize former poly layers (4/03)

In [30]:
# Getting paths of all remaining poly layers
xl_path = f'{workFiles_path}/versions/IGS_Tables_017.xlsx' # Last version of the Excel tables that had the poly layers
table = pd.read_excel(xl_path, sheet_name = 'Suitability factors')
poly_df = table.loc[table['type']=='poly']
poly_pointline_df = poly_df.loc[poly_df['Prepped Path'].str.contains('point|line', case = False, na = False)] # Separating point and line layers for different processing
poly_pointline_paths = poly_pointline_df['Prepped Path'].unique()
poly_only_df = poly_df.loc[~poly_df['Prepped Path'].str.contains('point|line', case = False, na = False)] # Inverse of the condition above
poly_only_paths = poly_only_df['Prepped Path'].unique()
# In the end because even on the poly layers many have very small geometries, I decide to process all with all_touched = True instead of separating points/lines and poly
poly_paths = poly_df['Prepped Path'].unique()
poly_paths

array(['C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/global_wind_2020_point_cropped.gpkg',
       'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/global_solar_2020_point_cropped.gpkg',
       'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/global_solar_2020_poly_cropped.gpkg',
       'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/OS_open_roads_lines.gpkg',
       'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/raw/Airports/hotosm_gbr_airports_points_gpkg.gpkg',
       'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/Study_area_basedOn_UK_BFC_EPSG27700.gpkg',
       'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent G

In [31]:
raster_path = f'{preppedDat_path}UKCEH_LC_2023_10m.tif'

for i, j in enumerate(poly_paths):
    logpath = f'{interimFiles_path}polyrast/logs/LastPolys_polyrast_{i}.txt'    
    polyrast(j, raster_path, logpath, all_touched=True)


Polyrast Start 2025-03-04 17:10:22
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/global_wind_2020_point_cropped.tif. 
 Processing....
Read ref raster Start 2025-03-04 17:10:22
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/UKCEH_LC_2023_10m.tif
Read poly Start 2025-03-04 17:10:22
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/global_wind_2020_point_cropped.gpkg
Rasterize poly Start 2025-03-04 17:10:23
Writing rasterized poly to file Start 2025-03-04 17:10:35
Polyrast End 2025-03-04 17:11:26
Polyras Code block run in 0:01:03.852487

Polyrast Start 2025-03-04 17:11:26
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/global_solar_2020_point_cropped.tif. 
 Proce

In [5]:
# redo solar poly because some poly were omitted in previous run
raster_path = f'{preppedDat_path}Land Cover/UKCEH_LC_2023_10m.tif'
logpath = f'{interimFiles_path}polyrast/logs/Solar_poly_polyrast.txt'
poly_path = f'{preppedDat_path}Existing RE/global_solar_2020_poly_cropped.gpkg'
polyrast(poly_path, raster_path, logpath, all_touched=True)


Polyrast Start 2025-03-07 17:48:02
File does not exist: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/global_solar_2020_poly_cropped.tif. 
 Processing....
Read ref raster Start 2025-03-07 17:48:02
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/Land Cover/UKCEH_LC_2023_10m.tif
Read poly Start 2025-03-07 17:48:03
 from: C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/data/prepped_data/Existing RE/global_solar_2020_poly_cropped.gpkg
Rasterize poly Start 2025-03-07 17:48:07
Writing rasterized poly to file Start 2025-03-07 17:48:45
Polyrast End 2025-03-07 17:50:15
Polyras Code block run in 0:02:12.873854


'C:/Users/roro_/Documents/University/UG year 3/6SSG0610 IGS Independent Geographical Study/output/intermediate_files/polyrast/global_solar_2020_poly_cropped.tif'

Second rasterization of global_solar_2020_poly_cropped wroekd even worse than the first, none of the poly were rasterized! Doing it in QGIS. Will need to check other layers for major discrepancies.

#### Merge poly-points layers of the same class

##### Existing Solar

In [7]:
write_path = f'{preppedDat_path}Existing RE/global_solar_2020_merged.tif'
path1 = f'{preppedDat_path}Existing RE/global_solar_2020_point_cropped.tif'
path2 = f'{preppedDat_path}Existing RE/global_solar_2020_poly_cropped.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 1-2
    logmsk = f'Merge 1-2\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_1 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_2) #Pixels in mask set to 1        
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 1-2
 Mask creation Start 2025-03-07 15:47:35
Merge Start 2025-03-07 15:47:36
2025-03-07 15:47:41 - Merge 1-2 complete.
2025-03-07 15:59:22 - Merge final result written to file.


##### Listed Buildings

In [8]:
write_path = f'{preppedDat_path}Heritage/Heritage_Listed_Buildings_merged.tif'
path1 = f'{preppedDat_path}Heritage/Heritage_Listed_Buildings_points.tif'
path2 = f'{preppedDat_path}Heritage/Heritage_Listed_Buildings_poly.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 1-2
    logmsk = f'Merge 1-2\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_1 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_2) #Pixels in mask set to 1        
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 1-2
 Mask creation Start 2025-03-07 16:00:33
Merge Start 2025-03-07 16:00:33
2025-03-07 16:00:38 - Merge 1-2 complete.
2025-03-07 16:11:18 - Merge final result written to file.


##### Water + Rivers

In [6]:
write_path = f'{preppedDat_path}Land Cover/Water_rivers_merged.tif'
path1 = f'{preppedDat_path}Land Cover/OS_open_rivers_lines.tif'
path2 = f'{preppedDat_path}Land Cover/UKCEH_10m_water.tif'

with rio.open(path1) as src_1:
    dask_ar_1 = da.from_array(src_1.read(1), chunks= 100)

with rio.open(path2) as src_2:
    dask_ar_2 = da.from_array(src_2.read(1), chunks= 100)

    # Merging 1-2
    logmsk = f'Merge 1-2\n Mask creation Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logmsk)
    mask = dask_ar_1 > 0 # Make a boolean array out of non-null raster pixels
    logdifst = f'Merge Start {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    print(logdifst)
    processed_data = da.where(mask, 1, dask_ar_2) #Pixels in mask set to 1        
    logdif = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge 1-2 complete.'
    print(logdif)
    
    # Write result to file
    with rio.open(
        write_path, 'w',
        driver='GTiff',
        height=processed_data.shape[0],
        width=processed_data.shape[1],
        count=1,
        dtype=processed_data.dtype,
        crs=src_1.crs,
        transform=src_1.transform,
        compress='DEFLATE'
    ) as dst:
        dst.write(processed_data.compute(), 1)

    logwri = f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - Merge final result written to file.'
    print(logwri)

Merge 1-2
 Mask creation Start 2025-03-07 18:29:55
Merge Start 2025-03-07 18:29:56
2025-03-07 18:29:56 - Merge 1-2 complete.
2025-03-07 18:54:42 - Merge final result written to file.
